# <center> <font style="color:rgb(100,109,254)">   Creating a Virtual Pen & Eraser </font> </center>

In this notebook you'll learn how to make a virtual marker and an eraser, this will let you draw with a colored marker on the air and it will draw it on the screen. Also you will be able to clear up the whole screen when the pen is really close to the screen, so this will act as a reset button.

Here's the Steps that are required for this:

1. *Find the color range of the target object and save it.*
2. *Apply the correct morphological operations to reduce noise.*
3. *Detect and track the object with contour detection.*
4. *Find the object's x,y location coordinates to draw on the screen.*
5. *Also add the Eraser functionality.*

In [28]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

## <font style="color:rgb(134,19,348)"> Finding Color range of target Pen and saving it   </font>
First and foremost we must find an appropriate color range for our target colored object , this range will be used in `cv2.inrange()` function to filter our object. We will also save our range array as `.npy` in our disk so we can access it later. Below function is the same as we have used in our color models notebook.

In [30]:
def nothing(x):
    pass
 
cap = cv2.VideoCapture(2,cv2.CAP_DSHOW)
cap.set(3,1280)
cap.set(4,720)

cv2.namedWindow("Trackbars")
 
cv2.createTrackbar("L - H", "Trackbars", 0, 179, nothing)
cv2.createTrackbar("L - S", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("L - V", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("U - H", "Trackbars", 179, 179, nothing)
cv2.createTrackbar("U - S", "Trackbars", 255, 255, nothing)
cv2.createTrackbar("U - V", "Trackbars", 255, 255, nothing)
 
 
while True:
    _, frame = cap.read()
    frame = cv2.flip( frame, 1 ) 

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
 
    l_h = cv2.getTrackbarPos("L - H", "Trackbars")
    l_s = cv2.getTrackbarPos("L - S", "Trackbars")
    l_v = cv2.getTrackbarPos("L - V", "Trackbars")
    u_h = cv2.getTrackbarPos("U - H", "Trackbars")
    u_s = cv2.getTrackbarPos("U - S", "Trackbars")
    u_v = cv2.getTrackbarPos("U - V", "Trackbars")
 
    # set the lower and upper range according to the value selected by the trackbar.
    lower_range = np.array([l_h, l_s, l_v])
    upper_range = np.array([u_h, u_s, u_v])
    
    # filter and get the binary mask, where white represents your target color.
    mask = cv2.inRange(hsv, lower_range, upper_range)
 
    # optionally you can also show the real part of the target color
    res = cv2.bitwise_and(frame, frame, mask=mask)
    
    mask_3 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    
    # stack all frames and show it
    stacked = np.hstack((mask_3,frame,res))
    cv2.imshow('Trackbars',cv2.resize(stacked,None,fx=0.4,fy=0.4))
 
    key = cv2.waitKey(1)
    if key == 27:
        break
        
    if key == ord('s'):
        thearray = [[l_h,l_s,l_v],[u_h, u_s, u_v]]
        print(thearray)
        
        # Also save this array as penval.npy
        np.save('media/m12/penval',thearray)
        break
    
cap.release()
cv2.destroyAllWindows()

[[112, 114, 122], [118, 255, 255]]


## <font style="color:rgb(134,19,348)"> Maximizing the Detection Mask and Getting rid of the noise   </font>
Now you may have noticed that there is some noise in the above program, this can easily be removed by morphological operations.

In [31]:
# This variable determines if we want to load color range from memory or use the ones defined in notebook. 
useload = True

# If true then load color range from memory
if useload:
    penval = np.load('media/m12/penval.npy')

cap = cv2.VideoCapture(2,cv2.CAP_DSHOW)
cap.set(3,1280)
cap.set(4,720)

# kernel for morphological operations
kernel = np.ones((5,5),np.uint8)

while(1):
    
    # Take each frame and flip it
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip( frame, 1 )

    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # If you're reading from memory then load the upper and lower ranges from there
    if useload:
            lower_range = penval[0]
            upper_range = penval[1]
            
    # Otherwise define your own custom values for upper and lower range.
    else:             
       lower_range  = np.array([26,80,147])
       upper_range = np.array([81,255,255])
    
    mask = cv2.inRange(hsv, lower_range, upper_range)
    
    # perform the morphological operations to get rid of the noise
    mask = cv2.erode(mask,kernel,iterations = 1)
    mask = cv2.dilate(mask,kernel,iterations = 2)

   
    res = cv2.bitwise_and(frame,frame, mask= mask)

    mask_3 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    
    # stack all frames and show it
    stacked = np.hstack((mask_3,frame,res))
    cv2.imshow('Trackbars',cv2.resize(stacked,None,fx=0.4,fy=0.4))
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break

cv2.destroyAllWindows()
cap.release()

## <font style="color:rgb(134,19,348)">  Tracking the Target Pen  </font>
Now that we have got a decent mask we can use it detect our pen using contour detection, in fact we will draw a bonding box around it.

In [32]:
# This variable determines if we want to load color range from memory or use the ones defined in notebook. 
useload = True

# If true then load color range from memory
if useload:
    penval = np.load('media/m12/penval.npy')

cap = cv2.VideoCapture(2,cv2.CAP_DSHOW)
cap.set(3,1280)
cap.set(4,720)

# kernel for morphological operations
kernel = np.ones((5,5),np.uint8)

# set the window to autosize so we can view this full screen.
cv2.namedWindow('image', cv2.WINDOW_NORMAL)

# this threshold is used to filter noise, the contour area must be bigger than this to qualify as an actual contour.
noiseth = 500

while(1):
    
    # Take each frame and flip it
    _, frame = cap.read()
    frame = cv2.flip( frame, 1 )

    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # If you're reading from memory then load the upper and lower ranges from there
    if useload:
            lower_range = penval[0]
            upper_range = penval[1]
            
    # Otherwise define your own custom values for upper and lower range.
    else:             
       lower_range  = np.array([26,80,147])
       upper_range = np.array([81,255,255])
    
    mask = cv2.inRange(hsv, lower_range, upper_range)
    
    # perform the morphological operations to get rid of the noise
    mask = cv2.erode(mask,kernel,iterations = 1)
    mask = cv2.dilate(mask,kernel,iterations = 2)
    
    # detect contour.
    contours, hierarchy = cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    # Make sure there was a contour present and also its size was bigger than some threshold.
    if contours and cv2.contourArea(max(contours, key = cv2.contourArea)) > noiseth:
        
        # grab the biggest contour
        c = max(contours, key = cv2.contourArea)
        
        # Draw a bounding box around it.
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)        

    cv2.imshow('image',frame)
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break

cv2.destroyAllWindows()
cap.release()

## <font style="color:rgb(134,19,348)"> Drawing with the Pen   </font>
Now that everything is set and we are easily able to track our target object, its time to use this object to draw on the frame, now we can easily do this by combining the above program with our mouse drawing on Gui program, so what we'll do is instead of drawing with the mouse by taking its x,y coordinates we will draw with the object by taking the corner x,y coordinates of our object. You can also track through the object's mid point.

In [34]:
useload = True
if useload:
    penval = np.load('media/m12/penval.npy')


cap = cv2.VideoCapture(2,cv2.CAP_DSHOW)
cap.set(3,1280)
cap.set(4,720)

kernel = np.ones((5,5),np.uint8)

# This is the canvas on which we will draw upon
canvas=None

# initilize x1,y1 points
x1,y1=0,0

# threshold for noise
noiseth = 500

while(1):
    # Take each frame and flip it
    _, frame = cap.read()
    frame = cv2.flip( frame, 1 )
    
    # initilize the canvas as a black image
    if canvas is None:
        canvas = np.zeros_like(frame)

    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # If you're reading from memory then load the upper and lower ranges from there
    if useload:
            lower_range = penval[0]
            upper_range = penval[1]
            
    # Otherwise define your own custom values for upper and lower range.
    else:             
       lower_range  = np.array([26,80,147])
       upper_range = np.array([81,255,255])
    
    mask = cv2.inRange(hsv, lower_range, upper_range)
    
    # perform the morphological operations to get rid of the noise
    mask = cv2.erode(mask,kernel,iterations = 1)
    mask = cv2.dilate(mask,kernel,iterations = 2)
    
    # detect contour.
    contours, hierarchy = cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    # Make sure there was a contour present and also its size was bigger than some threshold.
    if contours and cv2.contourArea(max(contours, key = cv2.contourArea)) > noiseth:
                
        c = max(contours, key = cv2.contourArea)    
        x2,y2,w,h = cv2.boundingRect(c)
        
        # if there were no previous points then save the detected x2,y2 as x1,y1. (logic similar to x1 = None)
        if x1 == 0 and y1 == 0:
            x1,y1= x2,y2
            
        else:
            # draw the line on the canvas
            canvas = cv2.line(canvas, (x1,y1),(x2,y2), [255,0,0], 4)
        
        # after the line is drawn the new points become the previous points.
        x1,y1= x2,y2

    else:
        # if there were no contours detected then make x1,y1 = 0
        x1,y1 =0,0
    
    # Merge the canvas and the frame.
    frame = cv2.add(frame,canvas)
    
    # Optionally stack both and show them, otherwise you just need to show the frame.
    stacked = np.hstack((canvas,frame))
    cv2.imshow('Trackbars',cv2.resize(stacked,None,fx=0.6,fy=0.6))

    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
        
    # When c is pressed clear the canvas
    if k == ord('c'):
        canvas = None

cv2.destroyAllWindows()
cap.release()

**One important thing to note is that you don't draw on frame but you draw on an empty canvas, a blank image, this is because a new frame is read every time in a loop, so what you draw now is lost in the next iteration**

## <font style="color:rgb(134,19,348)">  Adding the Eraser  </font>
In the above script we clear up the screen when the user presses `c` but lets automate this too, so one easy way we can do this is by detecting when the target object is too close to the camera and then if its too close we clear up the screen. (size of contour increases as it comes closer to the camera ). 

One other thing we will do is that we will also warn the user that we are about to clear the screen in a few seconds so he/she can take the object out of the frame.

In [36]:
import time

useload = True
if useload:
    penval = np.load('media/m12/penval.npy')

cap = cv2.VideoCapture(2,cv2.CAP_DSHOW)
cap.set(3,1280)
cap.set(4,720)

kernel = np.ones((5,5),np.uint8)

# making window size adjustable
cv2.namedWindow('image', cv2.WINDOW_NORMAL)

# This is the canvas on which we will draw upon
canvas=None

# initilize x1,y1 points
x1,y1=0,0

# threshold for noise
noiseth = 500

# threshold for eraser, the size of the contour must be bigger than for us to clear the canvas
eraserthresh = 40000

# A varaibel which tells when to clear canvas
clear = False

while(1):
    # Take each frame and flip it
    _, frame = cap.read()
    frame = cv2.flip( frame, 1 )
    
    # initilize the canvas as a black image
    if canvas is None:
        canvas = np.zeros_like(frame)

    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # If you're reading from memory then load the upper and lower ranges from there
    if useload:
            lower_range = penval[0]
            upper_range = penval[1]
            
    # Otherwise define your own custom values for upper and lower range.
    else:             
       lower_range  = np.array([26,80,147])
       upper_range = np.array([81,255,255])
    
    mask = cv2.inRange(hsv, lower_range, upper_range)
    
    # perform the morphological operations to get rid of the noise
    mask = cv2.erode(mask,kernel,iterations = 1)
    mask = cv2.dilate(mask,kernel,iterations = 2)
    
    # detect contour.
    contours, hierarchy = cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    # Make sure there was a contour present and also its size was bigger than some threshold.
    if contours and cv2.contourArea(max(contours, key = cv2.contourArea)) > noiseth:
                
        c = max(contours, key = cv2.contourArea)    
        x2,y2,w,h = cv2.boundingRect(c)
        
        # get the area of the contour
        area = cv2.contourArea(c)
        
        # if there were no previous points then save the detected x2,y2 as x1,y1. (logic similar to x1 = None)
        if x1 == 0 and y1 == 0:
            x1,y1= x2,y2
            
        else:
            # draw the line on the canvas
            canvas = cv2.line(canvas, (x1,y1),(x2,y2), [255,0,0], 5)
        
        # after the line is drawn the new points become the previous points.
        x1,y1= x2,y2
        
        # Now if the area is greater than the eraser threshold then set the clear variable to True
        if area > eraserthresh:
           cv2.putText(canvas,'Clearing Canvas',(100,200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 5, cv2.LINE_AA)
           clear = True 

    else:
        # if there were no contours detected then make x1,y1 = 0
        x1,y1 =0,0
    
   
    # Now this piece of code is just for smooth drawing. (Optional)
    _,mask = cv2.threshold(cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY), 20, 255, cv2.THRESH_BINARY)
    foreground = cv2.bitwise_and(canvas, canvas, mask = mask)
    background = cv2.bitwise_and(frame, frame, mask = cv2.bitwise_not(mask))
    frame = cv2.add(foreground,background)

    cv2.imshow('image',frame)
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
    
    # Clear the canvas after 2 second, if the clear variable is true
    if clear == True:
        time.sleep(1)
        canvas = None
        # and then set clear to false
        clear = False
        
cv2.destroyAllWindows()
cap.release()

###  <font style="color:rgb(34,169,134)"> Assignment: Add an actual Eraser to the above code </font>
Now you should add an additional feature to the above program, this feature will be an actual eraser which will be another colored object but instead of drawing it will erase the drawing on canvas. (Hint: black erases white)

In [ ]:
### ADD CODE HERE

